#### Step 0: Import packages and data

In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np

In [3]:
df = pd.read_csv('bg_LTO_search.csv')

# there are some entires with capitalized SearchMake and some in all lowercase, standardize the entries
df['SearchMake'] = df['SearchMake'].str.lower()
df['SearchModel'] = df['SearchModel'].str.lower()

# after standardizing the format, unify the entires
df=df.groupby(['SearchMake','SearchModel','Region','SearchCity']).agg({"Total_Searches":'sum'}).reset_index()
df.shape

(2055624, 5)

In [4]:
# Get rid of dirty data - SearchMake or SearchModel with multiple sections "|"

df_work = df[(~df.SearchMake.str.contains("\|",regex=True,na=False))|(~df.SearchModel.str.contains("\|",regex=True,na=False))].reset_index(drop=True)
df_work.shape

(1919626, 5)

In [5]:
#

df_work = df_work[(df_work.SearchMake != 'not used') & (df_work.SearchModel != 'not used')&(df_work.SearchCity != '(not set)')]
df_work.shape

(1398659, 5)

In [6]:
df_work = df_work.sort_values(by='Total_Searches',ascending=False).reset_index(drop=True)

#longer version
#df_work = df_work.sort_values(by='Total_Searches',ascending=False).reset_index(index=)
#df_work.drop('index',axis=1,inplace=True)

df_work.shape

(1398659, 5)

In [7]:
# code to look for provinces from the list
list_of_region = df.Region.unique().tolist()
[x for x in list_of_region if 'New B' in x]

['New Brunswick', 'West New Britain Province']

In [8]:
##### Useful code if you want to filter out non Canadian nor non American cities


canada = ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec','Newfoundland and Labrador',
          'Nova Scotia','Prince Edward Island','New Brunswick','Nunavut','Northwest Territories','Yukon']

#america = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
#  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
#  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
#  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
#  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
#  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
#  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
#  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

#df_work = df[(df.Region.isin(canada))|(df.Region.isin(america))] #filter the list by Canadian provinces
#df_work.shape

In [9]:
#Group 'Maritimes' and Territories and American States into 'Other'
Maritimes = ['Newfoundland and Labrador','Nova Scotia','Prince Edward Island','New Brunswick']
Provinces = ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec']

In [10]:
df_work.loc[df_work.Region.isin(Maritimes),'Province'] = 'Maritimes' #Assign Maritimes
df_work.loc[df_work.Region.isin(Provinces),'Province'] = df_work.Region
df_work['Province'].fillna('Others', inplace=True) #If all else, assign "Others"

### Alternative code based on if-else format
#df['name_match'] = df['First_name'].apply(lambda x: 'Match' if x == 'Bill' else 'Mis-Match')

#Try using .loc[row_indexer,col_indexer] = value instead
#https://www.dataquest.io/blog/settingwithcopywarning/

In [11]:
#Check to see if it's working

#df_work[df_work.Province.notnull()]
#df_work[df_work['Province'] == 'Alberta']
#df_work[(df_work['Province']!='Others')&(df_work['Province']!='Maritimes')].head(100)

In [12]:
# code to look for provinces from the list
list_of_city = df_work[df_work['Province']!='Others'].SearchCity.unique().tolist()
[x for x in list_of_city if 'Drayton Valley' in x]

['Drayton Valley']

In [13]:
df_work.shape

(1398659, 6)

In [14]:
###############################################################################

In [15]:
#del df_summary
df_summary = df_work.groupby("Province").agg({"Total_Searches":'sum'})
df_summary= df_summary.sort_values(by='Total_Searches',ascending=False).reset_index(drop=True)


df_summary['% of Total'] = (df_summary['Total_Searches']/ df_summary['Total_Searches'].sum()*100)
df_summary.loc['Total',:] = df_summary.sum().values


df_summary

### Alternative
#table['% of Total'] = (table.C / table.C.sum() * 100)
#table['% of B'] = (table.C / table.groupby(level=0).C.transform(sum) * 100)
#table.loc['total', :] = table.sum().values

#https://stackoverflow.com/questions/37148787/pandas-pivot-table-percent-calculations

,Total_Searches,% of Total
0,25393305.0,54.349163
1,8078830.0,17.291079
2,5494081.0,11.758954
3,4079673.0,8.731704
4,1485176.0,3.178715
5,918036.0,1.964868
6,711447.0,1.522706
7,561984.0,1.202812
Total,46722532.0,100.000000


In [16]:
canadian_cities = pd.read_csv('data_canadian_cities.csv')
canadian_cities.drop('location',axis=1,inplace=True)

canadian_cities.loc[canadian_cities['population']>= 200000,'Category'] = 'Metro'
canadian_cities.loc[(canadian_cities['population']>=50000)&(canadian_cities['population']<200000),'Category'] = 'Mid'
canadian_cities['Category'].fillna('Rural', inplace=True) #If all else, assign "Rural"


#canadian_cities.loc[(canadian_cities['population']>=50000) and (canadian_cities['population']<200000),'Category'] = 'Mid'
#canadian_cities['Category'].fillna('Rural', inplace=True) #If all else, assign "Others"

#canadian_cities[canadian_cities['Category']=='Rural'].sort_values(by='population',ascending=False)

In [17]:
canadian_cities.loc[canadian_cities.duplicated(subset=['asciiname'],keep=False), :]

#As you can see there are two entries of the same asciiname, drop first then merge

,asciiname,population,Category
204,Sainte-Catherine,16762,Rural
210,Sainte-Catherine,16211,Rural


In [18]:
canadian_cities.drop([210],inplace=True)
df_work2 = pd.merge(df_work,canadian_cities, left_on ='SearchCity',right_on='asciiname',how='left')
df_work2.shape

(1398659, 9)

In [19]:
df_work2[df_work2.Category.isnull()].Total_Searches.sum()/df_work2.Total_Searches.sum()
# nearly 10% of total searches are dropped from Category

0.09202037680663368

In [20]:
df_work2[(df_work2.Category.isnull())&(df_work2.Province!='Others')].Total_Searches.sum()/df_work2.Total_Searches.sum()
# nearly 6% of total searches from non-Other Provinces are dropped from Category

0.06201887774404007

In [21]:
### pick cities with more than 20,000 searches
list_prep = df_work2[(df_work2.Category.isnull())&(df_work2.Region.isin(canada))]
list_prep = list_prep.iloc[: , [3,4,5,8]]
list_prep = list_prep.groupby(['SearchCity','Province']).agg({"Total_Searches":'sum'}).sort_values(by=['Total_Searches'],ascending=False).reset_index()

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(list_prep)

In [22]:
list_prep.describe()

,Total_Searches
count,417.000000
mean,6959.258993
std,19389.546183
min,1.000000
25%,660.000000
50%,1757.000000
75%,4616.000000
max,177576.000000


In [23]:
list_prep = list_prep[list_prep['Total_Searches']>18000].reset_index() 
# make sure reset_index or groupby columns (Province, SearchCity) won't appear!!
list_prep = list_prep['SearchCity'].tolist()

#list_of_additional_cities
#for col in list_of_additional_cities.columns: 
#    print(col) 

df_work2.loc[df_work2['SearchCity'].isin(list_prep),'Category'] = 'Mid'
df_work2['Category'].fillna('Rural', inplace=True)

df_work2.shape

(1398659, 9)

In [24]:
#check to see any error values
df_work2[df_work2.Category.isnull()].shape

(0, 9)

In [25]:
df_final = df_work2.drop(['Region','SearchCity','asciiname','population'], axis=1)
#df_final.to_csv('result.csv')
df_final = df_final.groupby(['Province','Category','SearchMake','SearchModel']).agg({"Total_Searches":'sum'}).sort_values(by=['Total_Searches'],ascending=False).reset_index()
df_final.shape

(368069, 5)

In [26]:
df_final.to_csv('result.csv',index=False)

In [27]:
frames = {i:dat for i, dat in df_final.groupby(['Province','Category'])}

In [28]:
frames['Ontario','Metro'].head(5)

,Province,Category,SearchMake,SearchModel,Total_Searches
0,Ontario,Metro,bmw,3 series,342425
1,Ontario,Metro,ford,mustang,304794
2,Ontario,Metro,honda,civic,301238
3,Ontario,Metro,mercedes-benz,c-class,268657
4,Ontario,Metro,bmw,m,233715


In [29]:
df_dir = df_work2.drop(['SearchMake','SearchModel','Region','Total_Searches','population'], axis=1).drop_duplicates()
#df_dir.drop_duplicates()
df_dir_list = df_dir.SearchCity[(df_dir.Category !='Rural')&(df_dir.Province=='Ontario')].reset_index(drop=True)
df_dir_list.to_csv('result.csv',index=False)

C:\Users\skato\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [30]:
#import csv

#frames = {i:dat for i, dat in df_final.groupby('Province')}

# with open('dict.csv', 'w') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in frames.items():
#        writer.writerow([key, value])


In [31]:
# len(list_of_province)

#### SQL Lesson - always check there is no duplicate entry in joining tables!<br>
When I left joined df_work and canadian_cities (reference file) using df_work as the base table, row number increased from <br>
<br>
row # for df_work = 1,878,733 <br>
row # for canadian_cities = 400 <br>
<br>
Theoretically, row# for df_work2 should be same as df_work (because left joint), but df_work2 = 1,880,685<br>
1952 more rows in df_work2 than df_work!!!<br>
<br>
Two approaches were taken<br>
1. Check # of rows where [SearchCity] = isnull & [asciiname] = notnull - 67 rows <br>
df_work2[(df_work2['SearchCity'].isnull())&(df_work2['asciiname'].isnull())].shape<br><br>
2. Check for duplicates from reference table - 1 duplicate found on canadian_cities <br>
canadian_cities.loc[canadian_cities.duplicated(subset=['asciiname'],keep=False), :] <br>
<br>As you can tell from below, population number is different! manually delete one of the two!
<br><br>
The duplicated entry from canadian_cities created 1952 duplicated entries on df_work2 once joined<br>
df_work2.loc[df_work2.SearchCity =='Sainte-Catherine'].drop_duplicates(keep=False)